### What is Process and How Sub Process in Python Works

https://realpython.com/python-subprocess/

- Using a computer always involves processes

- We can explore the sub processes running inside the computer, using variety of tools. Psutil is one module in python that allows you to access the processes

- Processor is excellent at multi-tasking and process table / process control block is well organised

- Process can be interupted many times, however the processor knows where to start again

- Understanding how the processes are created inside the OS is important concept. Win32 API for windows system, and fork()/ exec() for unix system

- Every process will have PID, and typical resource usage like CPU %, RAM memory amount used 

![Alt text](image-1.png)

- Python subprocess module is for launching the child processes

- There are two function run() and Popen() class can be used for initiating the sub processes

### The subprocess module is mainly for calling programs other than Python. But, as you can see, you can call Python too if you want!

In [2]:
## The timer_process.py has been kept in the same folder, 
## which is called using the subprocess module

import subprocess

subprocess.run(["python","timer_process.py",'5']) #returncode is not accepted

Starting timer 5
.....Done!


CompletedProcess(args=['python', 'timer_process.py', '5'], returncode=0)

### Calling run() isn’t the same as calling programs on the command line. The run() function makes a system call, foregoing the need for a shell. 

In [3]:
import shlex

shlex.split("python timer_process.py 5")

['python', 'timer_process.py', '5']

In [5]:
subprocess.run(["vim"])

Vim: Warning: Output is not to a terminal
Vim: Warning: Input is not from a terminal


7[>4;2m=[>4;m[>4;2m                                                                               
_______________________________________________________                        
=========(_)===(_) (_______)==(_______|_______|_)(_)(_)                        
=================| |===================================                        
=========| |===| | | ||_|| |==|  ___)====| |==| || || |                        
========= \ \=/ /| | |===| |==| |========| |==| || || |                        
========== \___/=|_|_|===|_|==|_|========|_|===\_____/                         
                                                                               
               version : 2.3.0-dev   by : spacevim.org                                                                                                         

   [e]  <empty buffer>

   My most recently used files:

   [0]    ~/gitfolders/synapse/docker-compose.yml
   [1]    /etc/lightdm/lightdm.conf
   [2]    ~/gitfolders/freela

KeyboardInterrupt: 

In [7]:
#Subprocess returns once the process has been successfully completed. This 
#is a CompletedProcess object

completed = subprocess.run(shlex.split("python timer_process.py 10"))

Starting timer 10
..........Done!


In [10]:
completed.check_returncode()

In [11]:
complete = subprocess.run(['python','timer_process.py'], check=True)

usage: timer_process.py [-h] time
timer_process.py: error: the following arguments are required: time


CalledProcessError: Command '['python', 'timer_process.py']' returned non-zero exit status 2.

In [14]:
# There is no exception raised!!!
complete = subprocess.run(['python','timer_process.py'])

usage: timer_process.py [-h] time
timer_process.py: error: the following arguments are required: time


One thing to bear in mind is that the CalledProcessError does not apply to processes that may hang and block your execution indefinitely. To guard against that, you’d want to take advantage of the timeout parameter.

In [15]:
#TimeoutExpired for process that take too long 
subprocess.run(shlex.split("python timer_process.py 17"), timeout=5)

Starting timer 17
.....

TimeoutExpired: Command '['python', 'timer_process.py', '17']' timed out after 4.999984485999448 seconds

In [16]:
#File not found error

subprocess.run(["find_me.py"])

FileNotFoundError: [Errno 2] No such file or directory: 'find_me.py'

In [19]:
try:
    subprocess.run(
        ["python", "timer_process.py", "51"], timeout=10, check=True
    )
    #next file not found
except FileNotFoundError as exc:
    print(f"Process failed because the executable could not be found.\n{exc}")
    #called process error handling
except subprocess.CalledProcessError as exc:
    print(
        f"Process failed because did not return a successful return code. "
        f"Returned {exc.returncode}\n{exc}"
    )
    #Timeout error handling
except subprocess.TimeoutExpired as exc:
    print(f"Process timed out.\n{exc}")

Starting timer 51
..........Process timed out.
Command '['python', 'timer_process.py', '51']' timed out after 9.999987774004694 seconds


### Diving into Shell with Subprocess

There are actually two separate processes that make up the typical command-line experience:

The interpreter, which is typically thought of as the whole CLI. Common interpreters are Bash on Linux, Zsh on macOS, or PowerShell on Windows. In this tutorial, the interpreter will be referred to as the shell.

The interface, which displays the output of the interpreter in a window and sends user keystrokes to the interpreter. The interface is a separate process from the shell, sometimes called a terminal emulator.

While all new process are created with the same system calls, the context from which the system call is made is different. 

![Alt text](image-2.png)

The run() function can make a system call directly and doesn’t need to go through the shell to do so:

In [20]:
subprocess.run(["ls"])

api_requests_automation.ipynb
AutomatingXL.ipynb
biker_API.xls
biker_API.xlsx
boto_session_creation_configParser.ipynb
changeFileName.ipynb
example.conf
first_notebook.ipynb
image-1.png
image-2.png
image.png
Itertools_indepth.ipynb
openpyxl_glob_automation.ipynb
os_module_basics.ipynb
rope_checking_fbOPT.ipynb
sample.xlsx
serialized_json.txt
serialized_video_tutorial.txt
SubProcess_indepth.ipynb
testdata.xlsx
timer_process.py


CompletedProcess(args=['ls'], returncode=0)

The fact that many text-based programs can operate independently from the shell may make you wonder if you can cut out the middle process—namely, the shell—and use subprocess directly with the text-based programs typically associated with the shell.

Common reasons for using subprocess itself are similar in nature to using the shell with subprocess:

- When you have to use or analyze a black box, or even a white box

- When you want a wrapper for an application

- When you need to launch another application

- As an alternative to basic shell scripts


In [25]:
#Using subprocess with shell

subprocess.run(["bash","-c","ls /usr/bin | grep py3"])

dh_numpy3
f2py3
f2py3.11
html2markdown.py3
isympy3
py3clean
py3compile
py3rsa-decrypt
py3rsa-encrypt
py3rsa-keygen
py3rsa-priv2pub
py3rsa-sign
py3rsa-verify
py3versions
scapy3


CompletedProcess(args=['bash', '-c', 'ls /usr/bin | grep py3'], returncode=0)

In [26]:
subprocess.run(["ls /usr/bin/ | grep py3"], shell=True)

dh_numpy3
f2py3
f2py3.11
html2markdown.py3
isympy3
py3clean
py3compile
py3rsa-decrypt
py3rsa-encrypt
py3rsa-keygen
py3rsa-priv2pub
py3rsa-sign
py3rsa-verify
py3versions
scapy3


CompletedProcess(args=['ls /usr/bin/ | grep py3'], returncode=0)

To communicate with your process, you first should understand a little bit about how processes communicate in general, and then you’ll take a look at two examples to come to grips with the concepts.

When processes are initialized, there are three special streams that a process makes use of. A process does the following:

- Reads stdin for input

- Writes to stdout for general output

- Writes to stderr for error reporting

![process](image-3.png)


he subprocess fills up stdout and stderr, and **you fill up stdin**. Then you read the bytes in **stdout and stderr**, and the subprocess reads from stdin.

![Alt text](image-4.png)

**Magic number generator that outputs, well, a magic number.**

In [3]:
import subprocess

mag_num_proc = subprocess.run(["python","magic_no.py"])

645


In [4]:
mag_num_proc.stdout

In [10]:
mag_num_proc = subprocess.run(["python","magic_no.py"],capture_output=True)
mag_num_proc.stdout

b'435\n'

In [15]:
sum(
    int(
        subprocess.run(
            ["python",'magic_no.py'],capture_output=True
        ).stdout
    )
    for _ in range(2)
)

921

Processes communicate in bytes, and you have a few different ways to deal with encoding and decoding these bytes. Beneath the surface, subprocess has a few ways of getting into text mode.

In [16]:
mag_num_proc = subprocess.run(
    ["python",'magic_no.py'], capture_output=True, encoding='utf-8'
)

In [17]:
mag_num_proc.stdout

'797\n'

We have seen how to read and decode the output of a process, it’s time to take a look at writing to the input of a process.

In [19]:
proc = subprocess.run(
    ["python","reaction_game.py"],input='\n\n', encoding='utf-8'
)

Press enter to play
Ok, get ready!
go!
You reacted in 0 milliseconds!
Goodbye!


his is especially true if you want to wire up two processes together, feeding one stdout into another process’s stdin, for instance. In this section, you’ll be coming to grips with pipes and how to use them with the subprocess module.

In [20]:
mag_num_proc = subprocess.run(
    ["python","magic_no.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

In [28]:
#Pipe objects have limited capacity, so use files

from tempfile import TemporaryFile

with TemporaryFile() as f:
    ls_proc = subprocess.run(['python','magic_no.py'],stdout=f)
    f.seek(0)
    print(f.read().decode('utf-8'))

78



In [31]:
#Pipe using stdout, and the input between two subprocess run

ls_process = subprocess.run(['ls','/usr/bin'],stdout=subprocess.PIPE)
grep_process = subprocess.run(['grep','py3'],input=ls_process.stdout)
print(grep_process.stdout)

dh_numpy3
f2py3
f2py3.11
html2markdown.py3
isympy3
py3clean
py3compile
py3rsa-decrypt
py3rsa-encrypt
py3rsa-keygen
py3rsa-priv2pub
py3rsa-sign
py3rsa-verify
py3versions
scapy3
None


The name of Popen comes from a similar UNIX command that stands for pipe open. The command creates a pipe and then starts a new process that invokes the shell. The subprocess module, though, doesn’t automatically invoke the shell.

The run() function is a blocking function, which means that interacting dynamically with a process isn’t possible with it. However, the Popen() constructor starts a new process and continues, leaving the process running in parallel.

In [32]:
ls_process = subprocess.Popen(["ls", "/usr/bin"], stdout=subprocess.PIPE)
grep_process = subprocess.Popen(
    ["grep", "python"], stdin=ls_process.stdout, stdout=subprocess.PIPE
)

for line in grep_process.stdout:
    print(line.decode("utf-8").strip())

activate-global-python-argcomplete
apython
dh_python3-ply
ipython3
pybabel-python3
python
python2
python2.7
python3
python3.11
python3.11-config
python3-config
python3-futurize
python3-pasteurize
python3-qr
python-argcomplete-check-easy-install-script
python-argcomplete-tcsh
python-dotenv
python-faraday
register-python-argcomplete
x86_64-linux-gnu-python3.11-config
x86_64-linux-gnu-python3-config


The standard stream attributes of a CompletedProcess point to bytes objects or strings, but the same attributes of a Popen object point to the actual streams. This allows you to communicate with processes as they’re running.

In [ ]:
import subprocess

def get_char(process):
    character = process.stdout.read1(1)
    print(
        character.decode("utf-8"),
        end="",
        flush=True,  # Unbuffered print
    )
    return character.decode("utf-8")

def search_for_output(strings, process):
    buffer = ""
    while not any(string in buffer for string in strings):
        buffer = buffer + get_char(process)

with subprocess.Popen(
    [
        "python",
        "-u",  # Unbuffered stdout and stderr
        "reaction_game_v2.py",
    ],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
) as process:
    process.stdin.write(b"\n")
    process.stdin.flush()
    search_for_output(["==\n= ", "==\r\n= "], process)
    target_char = get_char(process)
    stdout, stderr = process.communicate(
        input=f"{target_char}\n".encode("utf-8"), timeout=10
    )
    print(stdout.decode("utf-8"))


Along the way, you’ve:

Learned about processes in general

Gone from basic to advanced usage of subprocess

Understood how to raise and handle errors when using run()

Gotten familiar with shells and their intricacies on both Windows and UNIX-like systems

Explored the use cases for subprocess through practical examples

Understood the standard I/O streams and how to interact with them

Come to grips with pipes, both in the shell and with subprocess

Looked at the Popen() constructor and used it for some advanced process communicatio